Read from music dataset and write it into 

In [11]:
!pip install pandas

In [22]:
# mkdir music if not exists
!mkdir -p ./raw/datasets/music
SEPERATOR = ";"

In [13]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

# add filter during loading
def getDF(path, limit=-1, filter=None):
  i = 0
  df = {}
  for d in parse(path):
    if(limit >0):
      if(i>=limit):
        break
    if(filter and filter(d)):
      #print(d["title"] + "|"+str(d["average_rating"])+"|"+str(d["rating_number"]))
      continue
    else:
      df[i] = d
      i += 1

  return pd.DataFrame.from_dict(df, orient='index')

In [14]:
def filterByRating(data, average_rating=3.5, rating_number=10):
    if(data['average_rating'] and  data['rating_number']):
        return data['average_rating']< average_rating or data['rating_number'] < rating_number
    else:
        return False

def filterByASIN(asin_list=[]):
    def filterInList(data):
        if(data['asin']):
            return (data['asin'] not in asin_list)
        else:
            return True
    return filterInList

def filterByParentASIN(asin_list=[]):
    def filterInList(data):
        if(data['parent_asin']):
            return (data['parent_asin'] not in asin_list)
        else:
            return True
    return filterInList

In [15]:
import math
def convertToNumber (s):
    return int.from_bytes(s.encode(), 'little')

def convertFromNumber (n):
    return n.to_bytes(math.ceil(n.bit_length() / 8), 'little').decode()

In [28]:
import hashlib
def hashToInt(s):
    return int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16) % (10 ** 4)

In [29]:
# Step 1: Read the json file into a DataFrame
# input_file = 'meta_CDs_and_Vinyl2018.json.gz'
input_file = './raw/datasets/meta_CDs_and_Vinyl2023.jsonl.gz'
# df = getDF(input_file, 100)
df = getDF(input_file, 1000, filterByRating) #enabled for 2023 data
df["music_id"] = df["parent_asin"].apply(hashToInt)
#TODO: check redudency of hash

#print(df.head())
# Filter rows where 'category' is non-empty or an empty array
# filtered_df = df[df['category'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
filtered_df = df[df['categories'].apply(lambda x: isinstance(x, list) and len(x) > 0)]#enabled for 2023 data
# Show the filtered DataFrame
print(filtered_df.count())

# convert id into int

# Convert the list in 'category' to a string with '|' as a separator
# filtered_df['genres'] = filtered_df['category'].apply(lambda x: '|'.join(map(str, x)))
filtered_df['genres'] = filtered_df['categories'].apply(lambda x: '|'.join(map(str, x)))#enabled for 2023 data
# Step 2: Pick necessary columns
# selected_content = ["asin", "title", "genres"]
selected_content = ["music_id", "title", "genres"]#enabled for 2023 data
df_content = filtered_df[selected_content]

# Step 3: Save the transformed DataFrame to a new CSV file with a custom separator and no header
# content_file = 'music/music2018.dat'
content_file = './raw/datasets/music/music2023.dat'

temp_content_file = content_file+'_temp'
df_content.to_csv(content_file, sep=SEPERATOR, index=False, header=False)
# with open(temp_content_file, 'r') as file:
#     file_data = file.read()
# file_data = file_data.replace(',', '::')
# with open(content_file, 'w') as file:
#     file.write(file_data)

main_category      1000
title              1000
average_rating     1000
rating_number      1000
features           1000
description        1000
price               856
images             1000
videos             1000
store               998
categories         1000
details            1000
parent_asin        1000
bought_together       0
subtitle              4
author                4
music_id           1000
dtype: int64


In [30]:
# Step 1: Read the json file into a DataFrame
# input_file = 'CDs_and_Vinyl2018.json.gz'
input_file = './raw/datasets/CDs_and_Vinyl2023.jsonl.gz'

# asin_list=filtered_df["asin"].to_list()
asin_list=filtered_df["parent_asin"].to_list()#enabled for 2023 data
count = filtered_df["rating_number"].sum()

# df = getDF(input_file,1000, filterByASIN(asin_list))
df = getDF(input_file,count, filterByParentASIN(asin_list))#enabled for 2023 data
# print(df.head())
df["music_id"] = df["parent_asin"].apply(hashToInt)

# TODO: sum the ratings by parent_asin to see the different in rating_number and actually ratings counted

# Step 2: Pick necessary columns
# selected_rating = ["asin", "user_id","rating","timestamp"]
selected_rating = ["user_id", "music_id", "rating","timestamp"] #enabled for 2023 data
df_rating = df[selected_rating]

# Step 3: Save the transformed DataFrame to a new CSV file with a custom separator and no header
# rating_file = 'music/rating2018.dat'
rating_file = './raw/datasets/music/rating2023.dat'
temp_rating_file = rating_file+'_temp'
df_rating.to_csv(rating_file, sep=SEPERATOR, index=False, header=False)
# with open(temp_rating_file, 'r') as file:
#     file_data = file.read()
# file_data = file_data.replace(',', '::')
# with open(rating_file, 'w') as file:
#     file.write(file_data)

In [ ]:
import os
RATINGS_FILE_NAME = "rating2023.dat"
MOVIES_FILE_NAME = "music2023.dat"
RATINGS_DATA_COLUMNS = ["UserID", "MovieID", "Rating", "Timestamp"]
MOVIES_DATA_COLUMNS = ["MovieID", "Title", "Genres"]
OUTPUT_TRAINING_DATA_FILENAME = "train_music_amz2023.tfrecord"
OUTPUT_TESTING_DATA_FILENAME = "test_music_amz2023.tfrecord"

# read the data 
ratings_df = pd.read_csv(
    os.path.join(os.curdir, "music",RATINGS_FILE_NAME),
    sep=SEPERATOR,
    names=RATINGS_DATA_COLUMNS,
    encoding="unicode_escape")

print(ratings_df.head())

movies_df = pd.read_csv(
    os.path.join(os.curdir, "music", MOVIES_FILE_NAME),
    sep=SEPERATOR,
    names=MOVIES_DATA_COLUMNS,
    encoding="unicode_escape")  # May contain unicode. Need to escape.

print(ratings_df.head())

                         UserID                   MovieID  Rating  \
0  AEVQ3KP55X4XECXWMHN6DHIDBYFQ  412271059740562891485250     5.0   
1  AFAIJYOUO3NAWLBDIKTQSC3DASWA  237470218963625094164546     5.0   
2  AE7BV6IMNPZ3F266H7PXMH3BZQNQ  355732224288405891657794     5.0   
3  AEUZHPIGXLWEQ7HW5LRYEPD3QYCQ  407493706522678071930946     5.0   
4  AER2M7E2IHDHZDVVXCFXTYKJ6EXQ  374399965658448707792962     5.0   

       Timestamp  
0  1406805139000  
1  1149955013000  
2  1637093229001  
3  1541959749242  
4  1578516789976  
                         UserID                   MovieID  Rating  \
0  AEVQ3KP55X4XECXWMHN6DHIDBYFQ  412271059740562891485250     5.0   
1  AFAIJYOUO3NAWLBDIKTQSC3DASWA  237470218963625094164546     5.0   
2  AE7BV6IMNPZ3F266H7PXMH3BZQNQ  355732224288405891657794     5.0   
3  AEUZHPIGXLWEQ7HW5LRYEPD3QYCQ  407493706522678071930946     5.0   
4  AER2M7E2IHDHZDVVXCFXTYKJ6EXQ  374399965658448707792962     5.0   

       Timestamp  
0  1406805139000  
1  1149955013000  